In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

##### Load the dataset

In [2]:
df = pd.read_csv('data.csv')

##### Fixing the dataset

Converting all retailer_codes to uppercase
Reasoning: 1 dataset has all codes in uppercase, and 1 has in lowercase. Likely an error.

In [76]:
df['retailer_code'] = df['retailer_code'].str.upper()

There are a lot of mos values which mean the same thing, but are represented with different codes. They are:
1.   DP                               Disputes
2.   DS                               Disputes
3.    
4.    PR                                Privacy
5.    PV                                Privacy
6.    
7.    RV                               Recovery
8.    CC                               Recovery
9.    
10.   Ls                     Report lost stolen
11.   LS                     Report lost stolen
12.   
13.  MR                    SMS Account Summary
14.  TA                    SMS Account Summary
15.  TS                    SMS Account Summary
16.  
17.  Tf                    SMS Flip Activation
18.  TF                    SMS Flip Activation
19.  
20.  TR                               Transfer
21.  ss                               Transfer
22.  
23.   ER                  eService Registration
24.   eR                  eService Registration

In [77]:
code_mapping = {
    'DS': 'DP', 'PV': 'PR', 'CC': 'RV', 'Ls': 'LS',
    'TA': 'TS', 'MR': 'TS', 'Tf': 'TF', 'ss': 'TR', 'eR': 'ER'
}

def normalize_codes(mos_string):
    codes = mos_string.split()
    normalized_codes = [code_mapping.get(code, code) for code in codes]
    return ' '.join(normalized_codes)

df['mos'] = df['mos'].apply(normalize_codes)

Print the columns and the first few rows of the dataset

In [ ]:
print("Columns are:")
print(df.columns)
print("First 2 rows are:")
print(df.head(2))

In [92]:
null_counts = df.isnull().sum()
print("Null/Empty counts for each column:")
print(null_counts)

Null/Empty counts for each column:
timestamp_call_key                        0
retailer_code                             0
serial                                    0
reason                                    0
mos                                       0
resolved                                  0
no_of_accounts_with_syf_13_march       3400
account_balance_13_march             113782
delinquency_history_13_march              0
account_open_date_13_march                0
account_status_13_march                   0
card_activation_status_13_march           0
eservice_ind_13_march                     0
ebill_enrolled_status_13_march            0
auto_pay_enrolled_status_13_march         0
no_of_accounts_with_syf_18_march       3402
account_balance_18_march             101829
delinquency_history_18_march              0
account_open_date_18_march                0
account_status_18_march                   0
card_activation_status_18_march           0
eservice_ind_18_march                    

In [89]:
columns_to_check = [
    'ebill_enrolled_status_13_march',
    'card_activation_status_13_march',
    'account_status_13_march'
]

for column in columns_to_check:
    unique_values = df[column].unique()
    print(f"Unique values in {column}: {unique_values}")


Unique values in ebill_enrolled_status_13_march: ['E' 'N' 'B']
Unique values in card_activation_status_13_march: ['0' '8' '7' '1']
Unique values in account_status_13_march: ['N' 'C' 'A' 'L' 'U' 'B']


In [84]:
df['ebill_enrolled_status_13_march'] = df['ebill_enrolled_status_13_march'].fillna('N').replace({' ': 'N', 'D': 'B', 'L': 'B'})
df['ebill_enrolled_status_18_march'] = df['ebill_enrolled_status_18_march'].fillna('N').replace({' ': 'N', 'D': 'B', 'L': 'B'})

In [86]:
df['card_activation_status_13_march'] = df['card_activation_status_13_march'].fillna('1').replace({' ': '1'})
df['card_activation_status_18_march'] = df['card_activation_status_18_march'].fillna('1').replace({' ': '1'})


In [88]:
df['account_status_13_march'] = df['account_status_13_march'].fillna('N').replace({'E': 'C', 'F': 'C', 'I': 'C', 'Z': 'C'})
df['account_status_18_march'] = df['account_status_18_march'].fillna('N').replace({'E': 'C', 'F': 'C', 'I': 'C', 'Z': 'C'})


In [90]:
df.to_csv('data_cleaned.csv', index=False)

Create Date and Time (of call) columns, from given timestamp

In [79]:
def parse_datetime(s):
    # Extract just the datetime part before the space
    datetime_part = s.split()[0]
    
    # Extract date and time parts
    date_part = datetime_part[:6]
    time_part = datetime_part[6:]
    
    # Parse date and time
    date_parsed = datetime.strptime(date_part, '%y%m%d').date()
    time_parsed = datetime.strptime(time_part, '%H%M%S').time()
    
    # Format the date in MM/DD/YYYY
    date_str = date_parsed.strftime('%m/%d/%Y')
    
    # Return both values
    return date_str, time_parsed

In [80]:
df['date_of_call'], df['time_of_call'] = zip(*df['timestamp_call_key'].apply(parse_datetime))

How many users called multiple times?

In [81]:
duplicate_serials_count = df['serial'].duplicated(keep=False).sum()
print(f"Count of serials that appear more than once: {duplicate_serials_count}")


Count of serials that appear more than once: 505263


Divide dataset depending on resolved status

In [91]:
# Filter and save rows with 'resolved' value
resolved_df = df[df['resolved'] == 'resolved']
resolved_df.to_csv('data_resolved.csv', index=False)

# Filter and save rows with 'floor' value
floor_df = df[df['resolved'] == 'floor']
floor_df.to_csv('data_floor.csv', index=False)


#### Retailer-wise Analysis

In [63]:
retailer_resolved_floor_counts = df.groupby(['retailer_code', 'resolved']).size().unstack(fill_value=0)
print(retailer_resolved_floor_counts)


resolved        floor  resolved
retailer_code                  
A              100384    317299
B               70128    296197
C               43032    172433
D               53403    212044
E               29013    129672
F               15948     62555
G               16388     63475
H                8125     38060
I                6654     38933
J                4757     26121
K                3052     16756
L                5869     14382
M                2157     12829
N                2561     13666
O                2262     10298
P                 765      3800
Q                 876      2214
R                 300      1120
S                 184       393
T                 191       141
U                 127        77
V                   3         0
W                  22        71
X                   7        26
Y                   9        19


In [51]:
# Counting occurrences of each resolution status for each retailer
resolution_counts = df.groupby('retailer_code')['resolved'].value_counts().unstack(fill_value=0)

# Calculating total calls per retailer for bubble size
resolution_counts['total'] = resolution_counts.sum(axis=1)

# Calculating percentage of 'floor' resolutions
resolution_counts['floor_percentage'] = (resolution_counts['floor'] / resolution_counts['total']) * 100

# Sorting retailers by floor percentage
resolution_counts_sorted = resolution_counts.sort_values(by='floor_percentage', ascending=False)

# Resetting index to use retailer_code as a column
resolution_counts_reset = resolution_counts_sorted.reset_index()

# Displaying the processed DataFrame for visualization
resolution_counts_reset


resolved,retailer_code,floor,resolved,total,floor_percentage
0,V,3,0,3,100.000000
1,U,127,77,204,62.254902
2,T,191,141,332,57.530120
3,Y,9,19,28,32.142857
4,S,184,393,577,31.889081
5,L,5869,14382,20251,28.981285
6,Q,876,2214,3090,28.349515
7,A,100384,317299,417683,24.033537
8,W,22,71,93,23.655914
9,X,7,26,33,21.212121


In [97]:
# Changes from 'account_status_13_march' to 'account_status_18_march'
account_status_changes = (df['account_status_13_march'] != df['account_status_18_march']).sum()

# Changes from 'ebill_enrolled_status_13_march' to 'ebill_enrolled_status_18_march'
ebill_enrolled_status_changes = (df['ebill_enrolled_status_13_march'] != df['ebill_enrolled_status_18_march']).sum()

# Changes from 'card_activation_status_13_march' to 'card_activation_status_18_march'
card_activation_status_changes = (df['card_activation_status_13_march'] != df['card_activation_status_18_march']).sum()

print(f"Account Status Changes: {account_status_changes}")
print(f"Ebill Enrolled Status Changes: {ebill_enrolled_status_changes}")
print(f"Card Activation Status Changes: {card_activation_status_changes}")

Account Status Changes: 34164
Ebill Enrolled Status Changes: 19762
Card Activation Status Changes: 35530


In [98]:
columns_to_group = [
    'account_status_13_march',
    'ebill_enrolled_status_13_march',
    'card_activation_status_13_march'
]

for column in columns_to_group:
    print(f"\nCounts for {column}:")
    group_counts = df.groupby([column, 'resolved']).size().unstack(fill_value=0)
    print(group_counts)



Counts for account_status_13_march:
resolved                  floor  resolved
account_status_13_march                  
A                          5061      1121
B                            42         4
C                         21236     36770
L                           683       747
N                        338998   1393770
U                           197       169

Counts for ebill_enrolled_status_13_march:
resolved                         floor  resolved
ebill_enrolled_status_13_march                  
B                                68053    273520
E                               136871    371518
N                               161293    787543

Counts for card_activation_status_13_march:
resolved                          floor  resolved
card_activation_status_13_march                  
0                                323271   1323005
1                                  5664     19610
7                                 19552     40070
8                                 17730    